# Developing an AI application

Going forward, AI algorithms will be incorporated into more and more everyday applications. For example, you might want to include an image classifier in a smart phone app. To do this, you'd use a deep learning model trained on hundreds of thousands of images as part of the overall application architecture. A large part of software development in the future will be using these types of models as common parts of applications. 

In this project, you'll train an image classifier to recognize different species of flowers. You can imagine using something like this in a phone app that tells you the name of the flower your camera is looking at. In practice you'd train this classifier, then export it for use in your application. We'll be using [this dataset](http://www.robots.ox.ac.uk/~vgg/data/flowers/102/index.html) of 102 flower categories, you can see a few examples below. 

<img src='assets/Flowers.png' width=500px>

The project is broken down into multiple steps:

* Load and preprocess the image dataset
* Train the image classifier on your dataset
* Use the trained classifier to predict image content

We'll lead you through each part which you'll implement in Python.

When you've completed this project, you'll have an application that can be trained on any set of labeled images. Here your network will be learning about flowers and end up as a command line application. But, what you do with your new skills depends on your imagination and effort in building a dataset. For example, imagine an app where you take a picture of a car, it tells you what the make and model is, then looks up information about it. Go build your own dataset and make something new.

First up is importing the packages you'll need. It's good practice to keep all the imports at the beginning of your code. As you work through this notebook and find you need to import a package, make sure to add the import up here.

In [1]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import matplotlib.pyplot as plt

import torch
import numpy as np
import torch.nn.functional as F
import torchvision

from torch import nn
from torch import optim
from torchvision import datasets, transforms, models
from collections import OrderedDict
from PIL import Image
from workspace_utils import active_session, keep_awake

from ipywidgets import FloatProgress
from IPython.display import display

## Load the data

Here you'll use `torchvision` to load the data ([documentation](http://pytorch.org/docs/0.3.0/torchvision/index.html)). The data should be included alongside this notebook, otherwise you can [download it here](https://s3.amazonaws.com/content.udacity-data.com/nd089/flower_data.tar.gz). The dataset is split into three parts, training, validation, and testing. For the training, you'll want to apply transformations such as random scaling, cropping, and flipping. This will help the network generalize leading to better performance. You'll also need to make sure the input data is resized to 224x224 pixels as required by the pre-trained networks.

The validation and testing sets are used to measure the model's performance on data it hasn't seen yet. For this you don't want any scaling or rotation transformations, but you'll need to resize then crop the images to the appropriate size.

The pre-trained networks you'll use were trained on the ImageNet dataset where each color channel was normalized separately. For all three sets you'll need to normalize the means and standard deviations of the images to what the network expects. For the means, it's `[0.485, 0.456, 0.406]` and for the standard deviations `[0.229, 0.224, 0.225]`, calculated from the ImageNet images.  These values will shift each color channel to be centered at 0 and range from -1 to 1.
 

In [2]:
data_dir = 'flowers'

data_transforms = { 
    
    'train': transforms.Compose([transforms.RandomRotation(30),
                transforms.RandomResizedCrop(224),
                transforms.RandomHorizontalFlip(),
                transforms.ToTensor(),
                transforms.Normalize([0.485, 0.456, 0.406],[0.229, 0.224, 0.225])]),

    'test': transforms.Compose([transforms.Resize(256),
                transforms.CenterCrop(224),
                transforms.ToTensor(),
                transforms.Normalize([0.485, 0.456, 0.406],[0.229, 0.224, 0.225])]),

    'valid': transforms.Compose([transforms.Resize(256),
                transforms.CenterCrop(224),
                transforms.ToTensor(),
                transforms.Normalize([0.485, 0.456, 0.406],[0.229, 0.224, 0.225])])
}

image_datasets = { x: datasets.ImageFolder('{}/{}'.format(data_dir,x), 
                                           transform=data_transforms[x]) 
                  for x in ['train','test','valid'] }

dataloaders = { x: torch.utils.data.DataLoader(image_datasets[x], 
                                               batch_size=64, shuffle=True)
               for x in ['train','test','valid'] }

dataset_sizes = {x: len(image_datasets[x]) for x in ['train','test','valid']}

### Label mapping

You'll also need to load in a mapping from category label to category name. You can find this in the file `cat_to_name.json`. It's a JSON object which you can read in with the [`json` module](https://docs.python.org/2/library/json.html). This will give you a dictionary mapping the integer encoded categories to the actual names of the flowers.

In [3]:
import json

with open('cat_to_name.json', 'r') as f:
    cat_to_name = json.load(f)


## Building and training the classifier

Now that the data is ready, it's time to build and train the classifier. As usual, you should use one of the pretrained models from `torchvision.models` to get the image features. Build and train a new feed-forward classifier using those features.

We're going to leave this part up to you. If you want to talk through it with someone, chat with your fellow students! You can also ask questions on the forums or join the instructors in office hours.

Refer to [the rubric](https://review.udacity.com/#!/rubrics/1663/view) for guidance on successfully completing this section. Things you'll need to do:

* Load a [pre-trained network](http://pytorch.org/docs/master/torchvision/models.html) (If you need a starting point, the VGG networks work great and are straightforward to use)
* Define a new, untrained feed-forward network as a classifier, using ReLU activations and dropout
* Train the classifier layers using backpropagation using the pre-trained network to get the features
* Track the loss and accuracy on the validation set to determine the best hyperparameters

We've left a cell open for you below, but use as many as you need. Our advice is to break the problem up into smaller parts you can run separately. Check that each part is doing what you expect, then move on to the next. You'll likely find that as you work through each part, you'll need to go back and modify your previous code. This is totally normal!

When training make sure you're updating only the weights of the feed-forward network. You should be able to get the validation accuracy above 70% if you build everything right. Make sure to try different hyperparameters (learning rate, units in the classifier, epochs, etc) to find the best model. Save those hyperparameters to use as default values in the next part of the project.

In [14]:
#creating model and assigning new classifier
def create_model(arch, hidden_units=512, dropout=0.4, learnrate=0.02):
    
    assert (arch in ['vgg16']), "Only vgg16 supported at this time"
    
    _models = { 'vgg16': models.vgg16(pretrained=True) }

    #freeze params, avoid backprop
    for param in _models[arch].parameters():
        param.requires_grad = False

    #hyperparameters for classifier
    input_size = 25088
    output_size = 102

    # Create the network, define the criterion and optimizer
    classifier = nn.Sequential(
        nn.Linear(input_size, hidden_units), 
        nn.ReLU(), 
        nn.Dropout(p=dropout),
        nn.Linear(hidden_units, output_size), 
        nn.LogSoftmax(dim=1))

    _models[arch].arch = arch
    
    #assign my classifier
    _models[arch].classifier = classifier
    _models[arch].class_to_idx = image_datasets['train'].class_to_idx
    _models[arch].idx_to_class = { x: y for y, x in _models[arch].class_to_idx.items() }
    
    _models[arch].optimizer = optim.Adam( _models[arch].classifier.parameters(), lr=learnrate)
    _models[arch].train_epochs = 0


    return _models[arch]

In [15]:
def train_model(model, optimizer, criterion=nn.NLLLoss(), epochs=5, device=torch.device("cuda:0" if torch.cuda.is_available() else "cpu")):
    
    model.to(device)
    
    for epoch in keep_awake(range(epochs)):
        
        print('-' * 40)
        print('Epoch {}/{}'.format(epoch+1, epochs))
        print('-' * 40)
    
        for phase in ['train', 'valid']:
            if phase == 'train':
                model.train()
            else:
                model.eval()
                
            running_loss = 0.0
            running_corrects = 0
            
            print("\n[{}-{}] Start | {} images".format(
                  epoch+1, phase, dataset_sizes[phase]))
            
            progbar = FloatProgress(min=0, max=dataset_sizes[phase])
            display(progbar)

            for step, (images, labels) in enumerate(dataloaders[phase]):

                images, labels = images.to(device), labels.to(device)

                #zero grad
                optimizer.zero_grad()
                    
                #forward
                with torch.set_grad_enabled(phase == 'train'):
                    
                    output = model.forward(images)            
                    loss = criterion(output, labels)
                    
                if phase == 'train':
                    loss.backward()
                    optimizer.step()
                
                _, preds = torch.max(output, dim=1)
                
                corrects = (preds == labels.data)
                corrects_sum = torch.sum(corrects)
               
                #print("[{}-{}-{}] Loss:{:.4f} ... Acc:{:.4f}".format(
                #            epoch+1, phase, step, 
                #            loss.item(),
                #            torch.mean(corrects.type(torch.FloatTensor))
                #            ))
                
                running_loss += loss.item() 
                running_corrects += corrects_sum
                
                progbar.value += images.size(0)

                
            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]
            
            model.train_epochs += 1
            
            progbar.layout.display = 'none'
            
            print("[{}-{}] End | Loss: {:.4f} ... Acc: {:.4f}\n".format(
                  epoch+1, phase, epoch_loss, epoch_acc))
            


In [6]:
def validate_model(model, criterion, device=torch.device("cuda:0" if torch.cuda.is_available() else "cpu")):
    
    model.to(device)
    model.eval()

    running_corrects = 0
    running_loss = 0
    
    phase = 'test'
    
    print("\n[{}] Start | {} images".format(
                  phase, dataset_sizes[phase]))
    
           
    progbar = FloatProgress(min=0, max=dataset_sizes[phase])
    display(progbar)
            
    for images, labels in dataloaders[phase]:
        
        images, labels = images.to(device), labels.to(device)
        
        with torch.set_grad_enabled(False):
            
            output = model.forward(images)
            loss = criterion(output, labels)
        
        _, preds = torch.max(output, dim=1)   
        running_loss += loss.item()
        running_corrects += torch.sum(preds == labels.data)
        
        progbar.value += images.size(0)
        
    test_loss = running_loss / dataset_sizes[phase]
    # Accuracy is number of correct predictions divided by all predictions, just take the mean
    accuracy = running_corrects.double() / dataset_sizes[phase]
    
    progbar.layout.display = 'none'
    
    print("[{}] End | Loss: {:.4f} ... Acc: {:.4f}\n".format(
                  phase, test_loss, accuracy))
    
    return test_loss, accuracy


In [16]:
# Training
learnrate = 0.002
epochs = 1

model_vgg16 = create_model(arch='vgg16')
optimizer = optim.Adam(model_vgg16.classifier.parameters(), lr=learnrate)
criterion=nn.NLLLoss()

train_model(model_vgg16, optimizer, criterion, epochs);

----------------------------------------
Epoch 1/1
----------------------------------------

[1-train] Start | 6552 images


FloatProgress(value=0.0, max=6552.0)

KeyboardInterrupt: 

## Testing your network

It's good practice to test your trained network on test data, images the network has never seen either in training or validation. This will give you a good estimate for the model's performance on completely new images. Run the test images through the network and measure the accuracy, the same way you did validation. You should be able to reach around 70% accuracy on the test set if the model has been trained well.

In [10]:
#Validation on the test set
criterion=nn.NLLLoss()
validate_model(model_vgg16, criterion);


[test] Start | 819 images


FloatProgress(value=0.0, max=819.0)

[test] End | Loss: 0.0218 ... Acc: 0.6300



## Save the checkpoint

Now that your network is trained, save the model so you can load it later for making predictions. You probably want to save other things such as the mapping of classes to indices which you get from one of the image datasets: `image_datasets['train'].class_to_idx`. You can attach this to the model as an attribute which makes inference easier later on.

```model.class_to_idx = image_datasets['train'].class_to_idx```

Remember that you'll want to completely rebuild the model later so you can use it for inference. Make sure to include any information you need in the checkpoint. If you want to load the model and keep training, you'll want to save the number of epochs as well as the optimizer state, `optimizer.state_dict`. You'll likely want to use this trained model in the next part of the project, so best to save it now.

In [12]:
def save_checkpoint(model, path):

    # Basic details
    checkpoint = {
        'arch': model.arch,
        'classifier': model.classifier,
        'state_dict': model.state_dict(),
        'optimizer' : model.optimizer,
        'optimizer_state_dict': model.optimizer.state_dict(),
        'class_to_idx': model.class_to_idx,
        'idx_to_class': model.idx_to_class,
        'train_epochs': model.train_epochs
    }

    # Save the data to the path
    torch.save(checkpoint, path)

In [13]:
#Save the checkpoint
save_checkpoint(model_vgg16,'vgg16-flowers-classifier.pth')

AttributeError: 'VGG' object has no attribute 'optimizer'

## Loading the checkpoint

At this point it's good to write a function that can load a checkpoint and rebuild the model. That way you can come back to this project and keep working on it without having to retrain the network.

In [ ]:
# Function that loads a checkpoint and rebuilds the model
def load_checkpoint(path):

    # Load in checkpoint
    checkpoint = torch.load(path)
    
    model = models.vgg16(pretrained=True)
        
    #freeze params, avoid backprop
    for param in model.parameters():
        param.requires_grad = False

    model.classifier = checkpoint['classifier']

    # Load state dict
    model.load_state_dict(checkpoint['state_dict'])

    # Model basics
    model.class_to_idx = checkpoint['class_to_idx']
    model.idx_to_cname = checkpoint['idx_to_cname']

    # Optimizer
    model.optimizer = checkpoint['optimizer']
    model.optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    
    return model

In [ ]:
model_vgg16 = load_checkpoint('vgg16' + checkpoint_path_suf)

# Inference for classification

Now you'll write a function to use a trained network for inference. That is, you'll pass an image into the network and predict the class of the flower in the image. Write a function called `predict` that takes an image and a model, then returns the top $K$ most likely classes along with the probabilities. It should look like 

```python
probs, classes = predict(image_path, model)
print(probs)
print(classes)
> [ 0.01558163  0.01541934  0.01452626  0.01443549  0.01407339]
> ['70', '3', '45', '62', '55']
```

First you'll need to handle processing the input image such that it can be used in your network. 

## Image Preprocessing

You'll want to use `PIL` to load the image ([documentation](https://pillow.readthedocs.io/en/latest/reference/Image.html)). It's best to write a function that preprocesses the image so it can be used as input for the model. This function should process the images in the same manner used for training. 

First, resize the images where the shortest side is 256 pixels, keeping the aspect ratio. This can be done with the [`thumbnail`](http://pillow.readthedocs.io/en/3.1.x/reference/Image.html#PIL.Image.Image.thumbnail) or [`resize`](http://pillow.readthedocs.io/en/3.1.x/reference/Image.html#PIL.Image.Image.thumbnail) methods. Then you'll need to crop out the center 224x224 portion of the image.

Color channels of images are typically encoded as integers 0-255, but the model expected floats 0-1. You'll need to convert the values. It's easiest with a Numpy array, which you can get from a PIL image like so `np_image = np.array(pil_image)`.

As before, the network expects the images to be normalized in a specific way. For the means, it's `[0.485, 0.456, 0.406]` and for the standard deviations `[0.229, 0.224, 0.225]`. You'll want to subtract the means from each color channel, then divide by the standard deviation. 

And finally, PyTorch expects the color channel to be the first dimension but it's the third dimension in the PIL image and Numpy array. You can reorder dimensions using [`ndarray.transpose`](https://docs.scipy.org/doc/numpy-1.13.0/reference/generated/numpy.ndarray.transpose.html). The color channel needs to be first and retain the order of the other two dimensions.

In [ ]:
def process_image(image_path):

    img = Image.open(image_path)

    shortdim = (0 if img.size[0] < img.size[1] else 1)

    newsize = [*img.size]
    newsize[shortdim] = 256
    ratio = (newsize[shortdim]/float(img.size[shortdim]))
    newsize[int(not(shortdim))] = int((float(img.size[int(not(shortdim))])*float(ratio)))

    img = img.resize(newsize)

    ccsize = 224

    left = (newsize[0] - ccsize) / 2
    top = (newsize[1] - ccsize) / 2
    right = (newsize[0] + ccsize) / 2
    bottom = (newsize[1] + ccsize) / 2

    img = img.crop((left, top, right, bottom))

    # Convert to numpy, transpose color dimension and normalize
    img = np.array(img).transpose((2, 0, 1)) / 256

    img = img - np.array([0.485, 0.456, 0.406]).reshape((3, 1, 1))
    img = img / np.array([0.229, 0.224, 0.225]).reshape((3, 1, 1))


    return torch.Tensor(img)


To check your work, the function below converts a PyTorch tensor and displays it in the notebook. If your `process_image` function works, running the output through this function should return the original image (except for the cropped out portions).

In [ ]:
def imshow(image, ax=None, title=None):
    """Imshow for Tensor."""
    if ax is None:
        fig, ax = plt.subplots()
    
    # PyTorch tensors assume the color channel is the first dimension
    # but matplotlib assumes is the third dimension
    image = image.numpy().transpose((1, 2, 0))
    
    # Undo preprocessing
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    image = std * image + mean
    
    # Image needs to be clipped between 0 and 1 or it looks like noise when displayed
    image = np.clip(image, 0, 1)
       
    ax.imshow(image)
    
    return ax

In [ ]:
imshow(process_image('flowers/test/1/image_06754.jpg'))
cat_to_name['1']

In [ ]:
images, labels = next(iter(dataloaders['train']))

imshow(images[0])

model_vgg16.idx_to_cname[labels[0]]


## Class Prediction

Once you can get images in the correct format, it's time to write a function for making predictions with your model. A common practice is to predict the top 5 or so (usually called top-$K$) most probable classes. You'll want to calculate the class probabilities then find the $K$ largest values.

To get the top $K$ largest values in a tensor use [`x.topk(k)`](http://pytorch.org/docs/master/torch.html#torch.topk). This method returns both the highest `k` probabilities and the indices of those probabilities corresponding to the classes. You need to convert from these indices to the actual class labels using `class_to_idx` which hopefully you added to the model or from an `ImageFolder` you used to load the data ([see here](#Save-the-checkpoint)). Make sure to invert the dictionary so you get a mapping from index to class as well.

Again, this method should take a path to an image and a model checkpoint, then return the probabilities and classes.

```python
probs, classes = predict(image_path, model)
print(probs)
print(classes)
> [ 0.01558163  0.01541934  0.01452626  0.01443549  0.01407339]
> ['70', '3', '45', '62', '55']
```

In [ ]:
def predict(image_path, model, topk=5):

    img_tensor = process_image(image_path)
    img_tensor.requires_grad_(False)
    
    #batch dimmension
    img_tensor.unsqueeze_(0)
    
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    
    img_tensor = img_tensor.to(device)
    
    with torch.set_grad_enabled(False):
        model.to(device)
        model.eval()
        output = model.forward(img_tensor)
        ps = torch.exp(output)
        
    return ps.topk(topk, dim=1)



In [ ]:
torch.cuda.is_available()

## Sanity Checking

Now that you can use a trained model for predictions, check to make sure it makes sense. Even if the testing accuracy is high, it's always good to check that there aren't obvious bugs. Use `matplotlib` to plot the probabilities for the top 5 classes as a bar graph, along with the input image. It should look like this:

<img src='assets/inference_example.png' width=300px>

You can convert from the class integer encoding to actual flower names with the `cat_to_name.json` file (should have been loaded earlier in the notebook). To show a PyTorch tensor as an image, use the `imshow` function defined above.

In [ ]:
def display_results(img, ps, labels):
    
    ps = ps.data.numpy().squeeze()

    fig, (ax1, ax2) = plt.subplots(figsize=(6,9), ncols=2)
    ax1.imshow(img.resize_(1, 28, 28).numpy().squeeze())
    ax1.axis('off')
    ax2.barh(np.arange(10), ps)
    ax2.set_aspect(0.1)
    ax2.set_yticks(np.arange(10))
   
       
    ax2.set_yticklabels(labels, size='small');
    
    ax2.set_title('Class Probability')
    ax2.set_xlim(0, 1.1)

    plt.tight_layout()

In [ ]:

imgpath = 'flowers/test/3/image_06641.jpg'

ps, classes = predict(imgpath, model_vgg16)


labels = [cat_to_name[model_vgg.class_to_idx[classes[x]]] for x in classes]

display_results(process_image(imgpath), ps, labels)
